In [14]:
import pandas as pd
import numpy as np
from collections import Counter
import time
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_validate, cross_val_predict, train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.under_sampling import CondensedNearestNeighbour, RandomUnderSampler
import operator
from joblib import dump, load

In [16]:
_data = pd.read_csv('./data/Modelar_UH2020.txt', sep='|')
_labels = _data.CLASE
_data.drop('CLASE', axis=1, inplace=True)
_data.drop('ID', axis=1, inplace=True)

# Variable categórica a numérica:
_data['CADASTRALQUALITYID'] = _data['CADASTRALQUALITYID'].map({'9': 0,
                               '8': 1,
                               '7': 2,
                               '6': 3,
                               '5': 4,
                               '4': 5,
                               '3': 6,
                               '2': 7,
                               '1': 8,
                               'C': 9,
                               'B': 10,
                               'A': 11})

# Imputación de NANs con la mediana:
_data['MAXBUILDINGFLOOR'].fillna(_data['MAXBUILDINGFLOOR'].median(), inplace=True)
_data['CADASTRALQUALITYID'].fillna(_data['CADASTRALQUALITYID'].median(), inplace=True)

# Normalización:
_data = pd.DataFrame( MinMaxScaler().fit_transform(_data), columns=_data.columns)
#_data = pd.DataFrame( StandardScaler().fit_transform(_data), columns=_data.columns)

In [132]:
X = _data.values
y = _labels.values

In [244]:
train, test, y_train, y_test = train_test_split(_data, _labels, test_size=0.1, random_state=123)

In [251]:
y_pred = OVO(train, y_train, test)

6000
FIT MODELS: DONE
PREDICT: DONE


In [252]:
# (TEST 0.1) UNDERSAMPLING 6000 , AGREGACIÓN: PROB VENCEDORA - PROB PERDEDORA:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

 AGRICULTURE       0.54      0.71      0.62        35
  INDUSTRIAL       0.48      0.72      0.58       426
      OFFICE       0.21      0.38      0.27       182
       OTHER       0.44      0.47      0.45       137
      PUBLIC       0.17      0.47      0.25       283
 RESIDENTIAL       0.97      0.87      0.92      9064
      RETAIL       0.25      0.29      0.26       196

    accuracy                           0.83     10323
   macro avg       0.44      0.56      0.48     10323
weighted avg       0.89      0.83      0.85     10323



In [250]:
# (TEST 0.1) Con todos los datos , AGREGACIÓN: PROB VENCEDORA - PROB PERDEDORA:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

 AGRICULTURE       0.77      0.69      0.73        35
  INDUSTRIAL       0.75      0.62      0.68       426
      OFFICE       0.70      0.16      0.27       182
       OTHER       0.75      0.36      0.49       137
      PUBLIC       0.54      0.17      0.26       283
 RESIDENTIAL       0.93      0.99      0.96      9064
      RETAIL       0.75      0.18      0.30       196

    accuracy                           0.92     10323
   macro avg       0.74      0.45      0.53     10323
weighted avg       0.90      0.92      0.90     10323



In [247]:
# (TEST 0.1) Con todos los datos , AGREGACIÓN: VOTO PONDERADO:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

 AGRICULTURE       0.79      0.66      0.72        35
  INDUSTRIAL       0.76      0.62      0.68       426
      OFFICE       0.67      0.16      0.26       182
       OTHER       0.74      0.39      0.51       137
      PUBLIC       0.52      0.17      0.26       283
 RESIDENTIAL       0.93      0.99      0.96      9064
      RETAIL       0.75      0.20      0.31       196

    accuracy                           0.92     10323
   macro avg       0.74      0.46      0.53     10323
weighted avg       0.90      0.92      0.90     10323



In [185]:
# (TEST 0.1) Con todos los datos:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

 AGRICULTURE       0.83      0.71      0.77        35
  INDUSTRIAL       0.77      0.62      0.69       426
      OFFICE       0.55      0.18      0.27       182
       OTHER       0.75      0.42      0.54       137
      PUBLIC       0.47      0.16      0.24       283
 RESIDENTIAL       0.93      0.99      0.96      9064
      RETAIL       0.75      0.21      0.33       196

    accuracy                           0.92     10323
   macro avg       0.72      0.47      0.54     10323
weighted avg       0.90      0.92      0.90     10323



In [182]:
# (TRAIN) Con todos los datos:
print(classification_report(_labels,y_pred))

              precision    recall  f1-score   support

 AGRICULTURE       1.00      1.00      1.00       338
  INDUSTRIAL       1.00      1.00      1.00      4490
      OFFICE       1.00      1.00      1.00      1828
       OTHER       1.00      1.00      1.00      1332
      PUBLIC       1.00      1.00      1.00      2976
 RESIDENTIAL       1.00      1.00      1.00     90173
      RETAIL       1.00      1.00      1.00      2093

    accuracy                           1.00    103230
   macro avg       1.00      1.00      1.00    103230
weighted avg       1.00      1.00      1.00    103230



In [127]:
# (TEST 0.2) Con todos los datos:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

 AGRICULTURE       0.83      0.69      0.75        80
  INDUSTRIAL       0.77      0.62      0.69       889
      OFFICE       0.55      0.16      0.25       382
       OTHER       0.70      0.44      0.54       269
      PUBLIC       0.56      0.21      0.31       598
 RESIDENTIAL       0.93      0.99      0.96     18018
      RETAIL       0.71      0.19      0.29       410

    accuracy                           0.92     20646
   macro avg       0.72      0.47      0.54     20646
weighted avg       0.90      0.92      0.90     20646



In [124]:
# (TEST 0.2) Con RandomUnderSampling para RESIDENTIAL : 6000
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

 AGRICULTURE       0.76      0.71      0.74        80
  INDUSTRIAL       0.48      0.71      0.57       889
      OFFICE       0.22      0.36      0.27       382
       OTHER       0.43      0.53      0.48       269
      PUBLIC       0.22      0.51      0.31       598
 RESIDENTIAL       0.97      0.88      0.92     18018
      RETAIL       0.25      0.27      0.26       410

    accuracy                           0.84     20646
   macro avg       0.48      0.57      0.51     20646
weighted avg       0.89      0.84      0.86     20646



In [77]:
def validacion_cruzada(_data, _labels, cv, under=True, n_us=6000):
    test_pred = _labels.copy()
    for train, test in cv.split(_data, _labels):
        y_pred = ovoModel(_data.take(train) ,_labels.take(train), _data.take(test), under, n_us)
        test_pred[test] = y_pred
        print(classification_report(y_pred, _labels.take(test), digits=4))
    return(test_pred)

In [78]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123456)
y_pred = validacion_cruzada(_data, _labels, skf, under=False)

FIT MODELS: DONE
PREDICT: DONE
              precision    recall  f1-score   support

 AGRICULTURE     0.6567    0.6875    0.6718        64
  INDUSTRIAL     0.6125    0.7736    0.6837       711
      OFFICE     0.1311    0.3750    0.1943       128
       OTHER     0.4286    0.7170    0.5365       159
      PUBLIC     0.2134    0.5799    0.3120       219
 RESIDENTIAL     0.9931    0.9297    0.9603     19264
      RETAIL     0.1766    0.7327    0.2846       101

    accuracy                         0.9138     20646
   macro avg     0.4589    0.6850    0.5205     20646
weighted avg     0.9570    0.9138    0.9317     20646

FIT MODELS: DONE
PREDICT: DONE
              precision    recall  f1-score   support

 AGRICULTURE     0.6567    0.7097    0.6822        62
  INDUSTRIAL     0.6258    0.7678    0.6896       732
      OFFICE     0.1311    0.4211    0.2000       114
       OTHER     0.4135    0.6667    0.5104       165
      PUBLIC     0.2269    0.5422    0.3199       249
 RESIDENTIAL    

In [79]:
# (CV-5 RF) FS-office, Agregación voto simple
print(classification_report(_labels,y_pred, digits=4))

              precision    recall  f1-score   support

 AGRICULTURE     0.7316    0.6775    0.7035       338
  INDUSTRIAL     0.7609    0.6229    0.6850      4490
      OFFICE     0.4425    0.1411    0.2140      1828
       OTHER     0.6688    0.3971    0.4984      1332
      PUBLIC     0.5478    0.2097    0.3033      2976
 RESIDENTIAL     0.9303    0.9933    0.9608     90173
      RETAIL     0.7351    0.1591    0.2616      2093

    accuracy                         0.9139    103230
   macro avg     0.6882    0.4573    0.5181    103230
weighted avg     0.8953    0.9139    0.8956    103230



In [ ]:
# (CV-5 KNN) ALL DATA, Agregación PROB MAX
print(classification_report(_labels,y_pred, digits=4))

# ko

In [73]:
# (CV-5 KNN) ALL DATA, Agregación SUMA PROBS
print(classification_report(_labels,y_pred, digits=4))

              precision    recall  f1-score   support

 AGRICULTURE     0.7211    0.5355    0.6146       338
  INDUSTRIAL     0.7616    0.5336    0.6276      4490
      OFFICE     0.4427    0.0908    0.1507      1828
       OTHER     0.7188    0.1689    0.2736      1332
      PUBLIC     0.6812    0.0948    0.1664      2976
 RESIDENTIAL     0.9130    0.9958    0.9526     90173
      RETAIL     0.7224    0.1343    0.2264      2093

    accuracy                         0.9040    103230
   macro avg     0.7087    0.3648    0.4303    103230
weighted avg     0.8844    0.9040    0.8770    103230



In [47]:
# (CV-5 RF) ALL DATA, Agregación PROB VENCEDORA-PERDEDORA
print(classification_report(_labels,y_pred, digits=4))

              precision    recall  f1-score   support

 AGRICULTURE     0.7434    0.6686    0.7040       338
  INDUSTRIAL     0.7589    0.6176    0.6810      4490
      OFFICE     0.4840    0.1242    0.1976      1828
       OTHER     0.6889    0.3641    0.4764      1332
      PUBLIC     0.5481    0.2127    0.3065      2976
 RESIDENTIAL     0.9281    0.9938    0.9598     90173
      RETAIL     0.7844    0.1443    0.2437      2093

    accuracy                         0.9131    103230
   macro avg     0.7051    0.4465    0.5099    103230
weighted avg     0.8953    0.9131    0.8937    103230



In [44]:
# (CV-5 RF) ALL DATA, Agregación suma prob
print(classification_report(_labels,y_pred, digits=4))

              precision    recall  f1-score   support

 AGRICULTURE     0.7516    0.6805    0.7143       338
  INDUSTRIAL     0.7607    0.6203    0.6834      4490
      OFFICE     0.4958    0.1302    0.2062      1828
       OTHER     0.6910    0.3761    0.4871      1332
      PUBLIC     0.5609    0.2151    0.3109      2976
 RESIDENTIAL     0.9287    0.9940    0.9603     90173
      RETAIL     0.7662    0.1472    0.2469      2093

    accuracy                         0.9138    103230
   macro avg     0.7079    0.4519    0.5156    103230
weighted avg     0.8962    0.9138    0.8948    103230



In [41]:
# (CV-5 RF) Con todos los datos, Agregación voto simple
print(classification_report(_labels,y_pred, digits=4))

              precision    recall  f1-score   support

 AGRICULTURE     0.7333    0.6834    0.7075       338
  INDUSTRIAL     0.7572    0.6196    0.6815      4490
      OFFICE     0.4525    0.1406    0.2145      1828
       OTHER     0.6730    0.3971    0.4995      1332
      PUBLIC     0.5529    0.2124    0.3069      2976
 RESIDENTIAL     0.9302    0.9932    0.9607     90173
      RETAIL     0.7284    0.1615    0.2644      2093

    accuracy                         0.9138    103230
   macro avg     0.6897    0.4583    0.5193    103230
weighted avg     0.8953    0.9138    0.8956    103230



In [38]:
# (CV-5 RF) Con todos los datos, Agregación voto simple, n_est=500
print(classification_report(_labels,y_pred, digits=4))

              precision    recall  f1-score   support

 AGRICULTURE     0.7363    0.6775    0.7057       338
  INDUSTRIAL     0.7578    0.6245    0.6847      4490
      OFFICE     0.4457    0.1346    0.2067      1828
       OTHER     0.6734    0.3979    0.5002      1332
      PUBLIC     0.5568    0.2124    0.3075      2976
 RESIDENTIAL     0.9303    0.9935    0.9608     90173
      RETAIL     0.7539    0.1610    0.2654      2093

    accuracy                         0.9141    103230
   macro avg     0.6935    0.4573    0.5187    103230
weighted avg     0.8959    0.9141    0.8958    103230



In [ ]:
# (CV-5 RF) Con todos los datos, Agregación voto simple, selección/creacción características
print(classification_report(_labels,y_pred, digits=4))

In [254]:
# (CV-5 RF) Con todos los datos, Agregación vencedora-perdedora
print(classification_report(_labels,y_pred, digits=4))

              precision    recall  f1-score   support

 AGRICULTURE     0.7500    0.6657    0.7053       338
  INDUSTRIAL     0.7588    0.6207    0.6828      4490
      OFFICE     0.4837    0.1214    0.1941      1828
       OTHER     0.6993    0.3649    0.4795      1332
      PUBLIC     0.5553    0.2161    0.3111      2976
 RESIDENTIAL     0.9282    0.9940    0.9600     90173
      RETAIL     0.7839    0.1438    0.2430      2093

    accuracy                         0.9135    103230
   macro avg     0.7084    0.4467    0.5108    103230
weighted avg     0.8958    0.9135    0.8941    103230



In [206]:
# (CV-3 RF) Con RandomUnderSampling para RESIDENTIAL : 6000
print(classification_report(_labels,y_pred, digits=4))

              precision    recall  f1-score   support

 AGRICULTURE     0.6475    0.7012    0.6733       338
  INDUSTRIAL     0.5110    0.7140    0.5957      4490
      OFFICE     0.2013    0.2938    0.2389      1828
       OTHER     0.4513    0.5083    0.4781      1332
      PUBLIC     0.2281    0.4903    0.3114      2976
 RESIDENTIAL     0.9628    0.8995    0.9301     90173
      RETAIL     0.2838    0.2408    0.2605      2093

    accuracy                         0.8499    103230
   macro avg     0.4694    0.5497    0.4983    103230
weighted avg     0.8871    0.8499    0.8652    103230



In [208]:
# (CV-5 RF) Con RandomUnderSampling para RESIDENTIAL : 6000
print(classification_report(_labels,y_pred, digits=4))

              precision    recall  f1-score   support

 AGRICULTURE     0.6343    0.7337    0.6804       338
  INDUSTRIAL     0.4988    0.7169    0.5883      4490
      OFFICE     0.1931    0.3184    0.2404      1828
       OTHER     0.4464    0.5248    0.4824      1332
      PUBLIC     0.2200    0.5148    0.3082      2976
 RESIDENTIAL     0.9659    0.8857    0.9240     90173
      RETAIL     0.2596    0.2680    0.2638      2093

    accuracy                         0.8399    103230
   macro avg     0.4597    0.5660    0.4982    103230
weighted avg     0.8883    0.8399    0.8597    103230



In [167]:
# (CV-5 RF) Con todos los datos
print(classification_report(_labels,y_pred, digits=4))

(34410,)

In [240]:
# (CV-5 RF) Con RandomUnderSampling para RESIDENTIAL : 3000
print(classification_report(_labels,y_pred, digits=4))

              precision    recall  f1-score   support

 AGRICULTURE     0.5558    0.7367    0.6336       338
  INDUSTRIAL     0.3698    0.7428    0.4938      4490
      OFFICE     0.1290    0.3627    0.1903      1828
       OTHER     0.3733    0.5405    0.4416      1332
      PUBLIC     0.1697    0.5685    0.2614      2976
 RESIDENTIAL     0.9750    0.7721    0.8618     90173
      RETAIL     0.1278    0.3244    0.1833      2093

    accuracy                         0.7455    103230
   macro avg     0.3858    0.5783    0.4380    103230
weighted avg     0.8842    0.7455    0.7967    103230



In [63]:
train, test, y_train, y_test = train_test_split(_data, _labels, test_size=0.20, random_state=123)

In [66]:
q = test[3:4]

In [67]:
y_test[3:4]

54227    PUBLIC
Name: CLASE, dtype: object

In [68]:
y_pred = ovoKNN(train ,y_train, q, False)

FIT MODELS: DONE
PREDICT: DONE
{'OFFICEvsRETAIL': array([[0.52134051, 0.47865949]]), 'OFFICEvsPUBLIC': array([[0.91537624, 0.08462376]]), 'OFFICEvsOTHER': array([[1., 0.]]), 'OFFICEvsINDUSTRIAL': array([[0.10397143, 0.89602857]]), 'OFFICEvsAGRICULTURE': array([[0., 1.]]), 'OFFICEvsRESIDENTIAL': array([[0.46966927, 0.53033073]]), 'AGRICULTUREvsRETAIL': array([[0., 1.]]), 'AGRICULTUREvsPUBLIC': array([[0., 1.]]), 'AGRICULTUREvsOTHER': array([[0., 1.]]), 'AGRICULTUREvsINDUSTRIAL': array([[0., 1.]]), 'AGRICULTUREvsRESIDENTIAL': array([[0., 1.]]), 'INDUSTRIALvsRETAIL': array([[0.22628609, 0.77371391]]), 'INDUSTRIALvsPUBLIC': array([[0.36682919, 0.63317081]]), 'INDUSTRIALvsOTHER': array([[0.59618888, 0.40381112]]), 'INDUSTRIALvsRESIDENTIAL': array([[0.09654287, 0.90345713]]), 'OTHERvsRETAIL': array([[0.16247058, 0.83752942]]), 'OTHERvsPUBLIC': array([[0.28206493, 0.71793507]]), 'OTHERvsRESIDENTIAL': array([[0., 1.]]), 'PUBLICvsRETAIL': array([[0.29840792, 0.70159208]]), 'PUBLICvsRESIDENTIAL'

In [69]:
y_pred

['OFFICE']

In [62]:
_labels[12:13]

12    PUBLIC
Name: CLASE, dtype: object

In [76]:
def ovoModel(_data, _labels, test, under=True, n_us=6000):
    # ====== DESCOMPOSICIÓN: ====== #
    # Creación de modelos one-vs-one:
    
    # OFFICE vs RETAIL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OFFICE"]
    data_2 = data[data.CLASE == "RETAIL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    
    data = data.drop('GEOM_R2', axis=1)
    
    data['MAXBUILDINGFLOOR2'] = data['MAXBUILDINGFLOOR']**2
    data['Q_G_3_0_32'] = data['Q_G_3_0_3']**2
    
    
    X = data.values
    y = labels.values
    model_office_vs_retail = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # OFFICE vs PUBLIC
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OFFICE"]
    data_2 = data[data.CLASE == "PUBLIC"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    
    data = data.drop('GEOM_R3', axis=1)
    
    data['Y2'] = data['Y']**2
    data['Q_NIR_8_0_92'] = data['Q_NIR_8_0_9']**2
    data['Q_B_2_0_32'] = data['Q_B_2_0_3']**2
    
    X = data.values
    y = labels.values
    model_office_vs_public = RandomForestClassifier(n_jobs=-1).fit(X,y)

    # OFFICE vs OTHER
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OFFICE"]
    data_2 = data[data.CLASE == "OTHER"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    
    data = data.drop('Q_B_2_0_2', axis=1)
    data = data.drop('Q_B_2_1_0', axis=1)
    data = data.drop('Q_NIR_8_0_8', axis=1)
    
    data['CADASTRALQUALITYID2'] = data['CADASTRALQUALITYID']**2
    data['Q_NIR_8_0_52'] = data['Q_NIR_8_0_5']**2
    data['Q_G_3_0_22'] = data['Q_G_3_0_2']**2
    
    X = data.values
    y = labels.values
    model_office_vs_other = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # OFFICE vs INDUSTRIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OFFICE"]
    data_2 = data[data.CLASE == "INDUSTRIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    
    data = data.drop('Q_NIR_8_0_8', axis=1)
    
    data['Q_B_2_0_62'] = data['Q_B_2_0_6']**2
    
    X = data.values
    y = labels.values
    model_office_vs_industrial = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # OFFICE vs AGRICULTURE
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OFFICE"]
    data_2 = data[data.CLASE == "AGRICULTURE"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    
    data = data.drop('Q_G_3_0_2', axis=1)
    
    data['Y2'] = data['Y']**2
    data['X2'] = data['X']**2
    data['Q_NIR_8_0_32'] = data['Q_NIR_8_0_3']**2
    
    X = data.values
    y = labels.values
    model_office_vs_agriculture = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # OFFICE vs RESIDENTIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OFFICE"]
    data_2 = data[data.CLASE == "RESIDENTIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    
    if(under):
        sampling_dict = {'RESIDENTIAL': n_us}
        us = RandomUnderSampler(random_state=42, sampling_strategy=sampling_dict) 
        X, y = us.fit_resample(X, y)
    
    model_office_vs_residential = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # AGRICULTURE vs RETAIL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "AGRICULTURE"]
    data_2 = data[data.CLASE == "RETAIL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    
    data = data.drop('GEOM_R4', axis=1)
    
    data['Q_B_2_0_92'] = data['Q_B_2_0_9']**2
    data['Q_G_3_0_12'] = data['Q_G_3_0_1']**2
    
    X = data.values
    y = labels.values
    model_agriculture_vs_retail = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # AGRICULTURE vs PUBLIC
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "AGRICULTURE"]
    data_2 = data[data.CLASE == "PUBLIC"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    
    data = data.drop('Q_G_3_1_0', axis=1)
    
    data['Q_NIR_8_0_62'] = data['Q_NIR_8_0_6']**2
    
    X = data.values
    y = labels.values
    model_agriculture_vs_public = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # AGRICULTURE vs OTHER
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "AGRICULTURE"]
    data_2 = data[data.CLASE == "OTHER"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    
    data = data.drop('Q_B_2_0_1', axis=1)
    
    data['X2'] = data['X']**2
    data['Q_B_2_0_92'] = data['Q_B_2_0_9']**2
    data['Q_G_3_0_12'] = data['Q_G_3_0_1']**2

    X = data.values
    y = labels.values
    model_agriculture_vs_other = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # AGRICULTURE vs INDUSTRIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "AGRICULTURE"]
    data_2 = data[data.CLASE == "INDUSTRIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_agriculture_vs_industrial = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # AGRICULTURE vs RESIDENTIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "AGRICULTURE"]
    data_2 = data[data.CLASE == "RESIDENTIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    
    if(under):
        sampling_dict = {'RESIDENTIAL': n_us}
        us = RandomUnderSampler(random_state=42, sampling_strategy=sampling_dict) 
        X, y = us.fit_resample(X, y)
    
    model_agriculture_vs_residential = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # INDUSTRIAL vs RETAIL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "INDUSTRIAL"]
    data_2 = data[data.CLASE == "RETAIL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_industrial_vs_retail = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # INDUSTRIAL vs PUBLIC
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "INDUSTRIAL"]
    data_2 = data[data.CLASE == "PUBLIC"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_industrial_vs_public = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # INDUSTRIAL vs OTHER
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "INDUSTRIAL"]
    data_2 = data[data.CLASE == "OTHER"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_industrial_vs_other = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # INDUSTRIAL vs RESIDENTIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "INDUSTRIAL"]
    data_2 = data[data.CLASE == "RESIDENTIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    
    if(under):
        sampling_dict = {'RESIDENTIAL': n_us}
        us = RandomUnderSampler(random_state=42, sampling_strategy=sampling_dict) 
        X, y = us.fit_resample(X, y)
    
    model_industrial_vs_residential = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # OTHER vs RETAIL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OTHER"]
    data_2 = data[data.CLASE == "RETAIL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_other_vs_retail = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # OTHER vs PUBLIC
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OTHER"]
    data_2 = data[data.CLASE == "PUBLIC"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_other_vs_public = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # OTHER vs RESIDENTIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OTHER"]
    data_2 = data[data.CLASE == "RESIDENTIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    
    if(under):
        sampling_dict = {'RESIDENTIAL': n_us}
        us = RandomUnderSampler(random_state=42, sampling_strategy=sampling_dict) 
        X, y = us.fit_resample(X, y)
    
    model_other_vs_residential = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # PUBLIC vs RETAIL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "PUBLIC"]
    data_2 = data[data.CLASE == "RETAIL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_public_vs_retail = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # PUBLIC vs RESIDENTIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "PUBLIC"]
    data_2 = data[data.CLASE == "RESIDENTIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    
    if(under):
        sampling_dict = {'RESIDENTIAL': n_us}
        us = RandomUnderSampler(random_state=42, sampling_strategy=sampling_dict) 
        X, y = us.fit_resample(X, y)
        
    model_public_vs_residential = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    # RETAIL vs RESIDENTIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "RETAIL"]
    data_2 = data[data.CLASE == "RESIDENTIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    
    if(under):
        sampling_dict = {'RESIDENTIAL': n_us}
        us = RandomUnderSampler(random_state=42, sampling_strategy=sampling_dict) 
        X, y = us.fit_resample(X, y)
    
    model_retail_vs_residential = RandomForestClassifier(n_jobs=-1).fit(X,y)
    
    print("FIT MODELS: DONE")
    
    # Diccionario que guarda las prob de cada modelo
    dict_prob = {}
    
    # Diccionario clases:
    dict_class = {
        'OFFICEvsRETAIL':['OFFICE','RETAIL'],
        'OFFICEvsPUBLIC':['OFFICE','PUBLIC'],
        'OFFICEvsOTHER':['OFFICE','OTHER'],
        'OFFICEvsINDUSTRIAL':['INDUSTRIAL','OFFICE'],
        'OFFICEvsAGRICULTURE':['AGRICULTURE','OFFICE'],
        'OFFICEvsRESIDENTIAL':['OFFICE','RESIDENTIAL'],
        'AGRICULTUREvsRETAIL':['AGRICULTURE','RETAIL'],
        'AGRICULTUREvsPUBLIC':['AGRICULTURE','PUBLIC'],
        'AGRICULTUREvsOTHER':['AGRICULTURE','OTHER'],
        'AGRICULTUREvsINDUSTRIAL':['AGRICULTURE','INDUSTRIAL'],
        'AGRICULTUREvsRESIDENTIAL':['AGRICULTURE','RESIDENTIAL'],
        'INDUSTRIALvsRETAIL':['INDUSTRIAL','RETAIL'],
        'INDUSTRIALvsPUBLIC':['INDUSTRIAL','PUBLIC'],
        'INDUSTRIALvsOTHER':['INDUSTRIAL','OTHER'],
        'INDUSTRIALvsRESIDENTIAL':['INDUSTRIAL','RESIDENTIAL'],
        'OTHERvsRETAIL':['OTHER','RETAIL'],
        'OTHERvsPUBLIC':['OTHER','PUBLIC'],
        'OTHERvsRESIDENTIAL':['OTHER','RESIDENTIAL'],
        'PUBLICvsRETAIL':['PUBLIC','RETAIL'],
        'PUBLICvsRESIDENTIAL':['PUBLIC','RESIDENTIAL'],
        'RETAILvsRESIDENTIAL':['RESIDENTIAL','RETAIL']
    }

    # Predicción de las probabilidades:

    # => Sin selección/creación de características:
    
#     dict_prob['OFFICEvsRETAIL'] = model_office_vs_retail.predict_proba(test.values)
#     dict_prob['OFFICEvsPUBLIC'] = model_office_vs_public.predict_proba(test.values)
#     dict_prob['OFFICEvsOTHER'] = model_office_vs_other.predict_proba(test.values)
#     dict_prob['OFFICEvsINDUSTRIAL'] = model_office_vs_industrial.predict_proba(test.values)
#     dict_prob['OFFICEvsAGRICULTURE'] = model_office_vs_agriculture.predict_proba(test.values)
#     dict_prob['OFFICEvsRESIDENTIAL'] = model_office_vs_residential.predict_proba(test.values)
#     dict_prob['AGRICULTUREvsRETAIL'] = model_agriculture_vs_retail.predict_proba(test.values)
#     dict_prob['AGRICULTUREvsPUBLIC'] = model_agriculture_vs_public.predict_proba(test.values)
#     dict_prob['AGRICULTUREvsOTHER'] = model_agriculture_vs_other.predict_proba(test.values)
#     dict_prob['AGRICULTUREvsINDUSTRIAL'] = model_agriculture_vs_industrial.predict_proba(test.values)
#     dict_prob['AGRICULTUREvsRESIDENTIAL'] = model_agriculture_vs_residential.predict_proba(test.values)
#     dict_prob['INDUSTRIALvsRETAIL'] = model_industrial_vs_retail.predict_proba(test.values)
#     dict_prob['INDUSTRIALvsPUBLIC'] = model_industrial_vs_public.predict_proba(test.values)
#     dict_prob['INDUSTRIALvsOTHER'] = model_industrial_vs_other.predict_proba(test.values)
#     dict_prob['INDUSTRIALvsRESIDENTIAL'] = model_industrial_vs_residential.predict_proba(test.values)
#     dict_prob['OTHERvsRETAIL'] = model_other_vs_retail.predict_proba(test.values)
#     dict_prob['OTHERvsPUBLIC'] = model_other_vs_public.predict_proba(test.values)
#     dict_prob['OTHERvsRESIDENTIAL'] = model_other_vs_residential.predict_proba(test.values)
#     dict_prob['PUBLICvsRETAIL'] = model_public_vs_retail.predict_proba(test.values)
#     dict_prob['PUBLICvsRESIDENTIAL'] = model_public_vs_residential.predict_proba(test.values)
#     dict_prob['RETAILvsRESIDENTIAL'] = model_retail_vs_residential.predict_proba(test.values)

    # => Con selección/creación de características:

    aux = test.copy()
    aux = aux.drop('GEOM_R2', axis=1)
    aux['MAXBUILDINGFLOOR2'] = aux['MAXBUILDINGFLOOR']**2
    aux['Q_G_3_0_32'] = aux['Q_G_3_0_3']**2
    dict_prob['OFFICEvsRETAIL'] = model_office_vs_retail.predict_proba(aux.values)

    aux = test.copy()
    aux = aux.drop('GEOM_R3', axis=1)
    aux['Y2'] = aux['Y']**2
    aux['Q_NIR_8_0_92'] = aux['Q_NIR_8_0_9']**2
    aux['Q_B_2_0_32'] = aux['Q_B_2_0_3']**2
    dict_prob['OFFICEvsPUBLIC'] = model_office_vs_public.predict_proba(aux.values)

    aux = test.copy()
    aux = aux.drop('Q_B_2_0_2', axis=1)
    aux = aux.drop('Q_B_2_1_0', axis=1)
    aux = aux.drop('Q_NIR_8_0_8', axis=1)
    aux['CADASTRALQUALITYID2'] = aux['CADASTRALQUALITYID']**2
    aux['Q_NIR_8_0_52'] = aux['Q_NIR_8_0_5']**2
    aux['Q_G_3_0_22'] = aux['Q_G_3_0_2']**2
    dict_prob['OFFICEvsOTHER'] = model_office_vs_other.predict_proba(aux.values)

    aux = test.copy()
    aux = aux.drop('Q_NIR_8_0_8', axis=1)
    aux['Q_B_2_0_62'] = aux['Q_B_2_0_6']**2
    dict_prob['OFFICEvsINDUSTRIAL'] = model_office_vs_industrial.predict_proba(aux.values)

    aux = test.copy()
    aux = aux.drop('Q_G_3_0_2', axis=1)
    aux['Y2'] = aux['Y']**2
    aux['X2'] = aux['X']**2
    aux['Q_NIR_8_0_32'] = aux['Q_NIR_8_0_3']**2
    dict_prob['OFFICEvsAGRICULTURE'] = model_office_vs_agriculture.predict_proba(aux.values)

    dict_prob['OFFICEvsRESIDENTIAL'] = model_office_vs_residential.predict_proba(test.values)

    dict_prob['AGRICULTUREvsRETAIL'] = model_agriculture_vs_retail.predict_proba(test.values)

    dict_prob['AGRICULTUREvsPUBLIC'] = model_agriculture_vs_public.predict_proba(test.values)

    dict_prob['AGRICULTUREvsOTHER'] = model_agriculture_vs_other.predict_proba(test.values)

    dict_prob['AGRICULTUREvsINDUSTRIAL'] = model_agriculture_vs_industrial.predict_proba(test.values)

    dict_prob['AGRICULTUREvsRESIDENTIAL'] = model_agriculture_vs_residential.predict_proba(test.values)

    dict_prob['INDUSTRIALvsRETAIL'] = model_industrial_vs_retail.predict_proba(test.values)

    dict_prob['INDUSTRIALvsPUBLIC'] = model_industrial_vs_public.predict_proba(test.values)

    dict_prob['INDUSTRIALvsOTHER'] = model_industrial_vs_other.predict_proba(test.values)

    dict_prob['INDUSTRIALvsRESIDENTIAL'] = model_industrial_vs_residential.predict_proba(test.values)

    dict_prob['OTHERvsRETAIL'] = model_other_vs_retail.predict_proba(test.values)

    dict_prob['OTHERvsPUBLIC'] = model_other_vs_public.predict_proba(test.values)

    dict_prob['OTHERvsRESIDENTIAL'] = model_other_vs_residential.predict_proba(test.values)

    dict_prob['PUBLICvsRETAIL'] = model_public_vs_retail.predict_proba(test.values)

    dict_prob['PUBLICvsRESIDENTIAL'] = model_public_vs_residential.predict_proba(test.values)

    dict_prob['RETAILvsRESIDENTIAL'] = model_retail_vs_residential.predict_proba(test.values)

    print("PREDICT: DONE")
    
    # ====== AGREGACIÓN: ====== #
    pred = []
    
    for i in range(0, test.shape[0]):

        dict_count = {
            'OFFICE': 0,
            'AGRICULTURE': 0,
            'INDUSTRIAL': 0,
            'OTHER': 0,
            'PUBLIC': 0,
            'RETAIL': 0,
            'RESIDENTIAL': 0
        }

        for key in dict_prob.keys():
            
            # ==> VOTO SIMPLE:
            if (dict_prob[key][i][0]>dict_prob[key][i][1]):
                dict_count[dict_class[key][0]] +=1
            else:
                dict_count[dict_class[key][1]] +=1

            # ==> SUMA PROB DE CADA CLASE:
#             dict_count[dict_class[key][0]] += dict_prob[key][i][0]
#             dict_count[dict_class[key][1]] += dict_prob[key][i][1]

            # ==> PROB VENCEDORA - PROB PERDEDORA:
#             if (dict_prob[key][i][0]>dict_prob[key][i][1]):
#                 dict_count[dict_class[key][0]] += dict_prob[key][i][0]-dict_prob[key][i][1]
#             else:
#                 dict_count[dict_class[key][1]] += dict_prob[key][i][1]-dict_prob[key][i][0]
            
#             # ==> PROB MAX:
#             if (dict_prob[key][i][0]>dict_prob[key][i][1]):
#                 dict_count[dict_class[key][0]] += dict_prob[key][i][0]-dict_prob[key][i][1]
#             else:
#                 dict_count[dict_class[key][1]] += dict_prob[key][i][1]-dict_prob[key][i][0]

        predicted_class = max(dict_count.items(), key=operator.itemgetter(1))[0]
        pred.append(predicted_class)
    
    return(pred)

In [71]:
def ovoKNN(_data, _labels, test, under=True, n_us=6000):
    # ====== DESCOMPOSICIÓN: ====== #
    # Creación de modelos one-vs-one:
    
    # OFFICE vs RETAIL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OFFICE"]
    data_2 = data[data.CLASE == "RETAIL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    
#     data = data.drop('GEOM_R2', axis=1)
    
#     data['MAXBUILDINGFLOOR2'] = data['MAXBUILDINGFLOOR']**2
#     data['Q_G_3_0_32'] = data['Q_G_3_0_3']**2
    
    
    X = data.values
    y = labels.values
    model_office_vs_retail = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # OFFICE vs PUBLIC
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OFFICE"]
    data_2 = data[data.CLASE == "PUBLIC"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    
#     data = data.drop('GEOM_R3', axis=1)
    
#     data['Y2'] = data['Y']**2
#     data['Q_NIR_8_0_92'] = data['Q_NIR_8_0_9']**2
#     data['Q_B_2_0_32'] = data['Q_B_2_0_3']**2
    
    X = data.values
    y = labels.values
    model_office_vs_public = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)

    # OFFICE vs OTHER
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OFFICE"]
    data_2 = data[data.CLASE == "OTHER"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    
#     data = data.drop('Q_B_2_0_2', axis=1)
#     data = data.drop('Q_B_2_1_0', axis=1)
#     data = data.drop('Q_NIR_8_0_8', axis=1)
    
#     data['CADASTRALQUALITYID2'] = data['CADASTRALQUALITYID']**2
#     data['Q_NIR_8_0_52'] = data['Q_NIR_8_0_5']**2
#     data['Q_G_3_0_22'] = data['Q_G_3_0_2']**2
    
    X = data.values
    y = labels.values
    model_office_vs_other = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # OFFICE vs INDUSTRIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OFFICE"]
    data_2 = data[data.CLASE == "INDUSTRIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    
#     data = data.drop('Q_NIR_8_0_8', axis=1)
    
#     data['Q_B_2_0_62'] = data['Q_B_2_0_6']**2
    
    X = data.values
    y = labels.values
    model_office_vs_industrial = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # OFFICE vs AGRICULTURE
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OFFICE"]
    data_2 = data[data.CLASE == "AGRICULTURE"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    
#     data = data.drop('Q_G_3_0_2', axis=1)
    
#     data['Y2'] = data['Y']**2
#     data['X2'] = data['X']**2
#     data['Q_NIR_8_0_32'] = data['Q_NIR_8_0_3']**2
    
    X = data.values
    y = labels.values
    model_office_vs_agriculture = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # OFFICE vs RESIDENTIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OFFICE"]
    data_2 = data[data.CLASE == "RESIDENTIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    
    if(under):
        sampling_dict = {'RESIDENTIAL': n_us}
        us = RandomUnderSampler(random_state=42, sampling_strategy=sampling_dict) 
        X, y = us.fit_resample(X, y)
    
    model_office_vs_residential = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # AGRICULTURE vs RETAIL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "AGRICULTURE"]
    data_2 = data[data.CLASE == "RETAIL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_agriculture_vs_retail = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # AGRICULTURE vs PUBLIC
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "AGRICULTURE"]
    data_2 = data[data.CLASE == "PUBLIC"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_agriculture_vs_public = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # AGRICULTURE vs OTHER
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "AGRICULTURE"]
    data_2 = data[data.CLASE == "OTHER"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_agriculture_vs_other = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # AGRICULTURE vs INDUSTRIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "AGRICULTURE"]
    data_2 = data[data.CLASE == "INDUSTRIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_agriculture_vs_industrial = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # AGRICULTURE vs RESIDENTIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "AGRICULTURE"]
    data_2 = data[data.CLASE == "RESIDENTIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    
    if(under):
        sampling_dict = {'RESIDENTIAL': n_us}
        us = RandomUnderSampler(random_state=42, sampling_strategy=sampling_dict) 
        X, y = us.fit_resample(X, y)
    
    model_agriculture_vs_residential = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # INDUSTRIAL vs RETAIL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "INDUSTRIAL"]
    data_2 = data[data.CLASE == "RETAIL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_industrial_vs_retail = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # INDUSTRIAL vs PUBLIC
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "INDUSTRIAL"]
    data_2 = data[data.CLASE == "PUBLIC"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_industrial_vs_public = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # INDUSTRIAL vs OTHER
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "INDUSTRIAL"]
    data_2 = data[data.CLASE == "OTHER"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_industrial_vs_other = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # INDUSTRIAL vs RESIDENTIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "INDUSTRIAL"]
    data_2 = data[data.CLASE == "RESIDENTIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    
    if(under):
        sampling_dict = {'RESIDENTIAL': n_us}
        us = RandomUnderSampler(random_state=42, sampling_strategy=sampling_dict) 
        X, y = us.fit_resample(X, y)
    
    model_industrial_vs_residential = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # OTHER vs RETAIL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OTHER"]
    data_2 = data[data.CLASE == "RETAIL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_other_vs_retail = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # OTHER vs PUBLIC
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OTHER"]
    data_2 = data[data.CLASE == "PUBLIC"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_other_vs_public = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # OTHER vs RESIDENTIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "OTHER"]
    data_2 = data[data.CLASE == "RESIDENTIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    
    if(under):
        sampling_dict = {'RESIDENTIAL': n_us}
        us = RandomUnderSampler(random_state=42, sampling_strategy=sampling_dict) 
        X, y = us.fit_resample(X, y)
    
    model_other_vs_residential = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # PUBLIC vs RETAIL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "PUBLIC"]
    data_2 = data[data.CLASE == "RETAIL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    model_public_vs_retail = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # PUBLIC vs RESIDENTIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "PUBLIC"]
    data_2 = data[data.CLASE == "RESIDENTIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    
    if(under):
        sampling_dict = {'RESIDENTIAL': n_us}
        us = RandomUnderSampler(random_state=42, sampling_strategy=sampling_dict) 
        X, y = us.fit_resample(X, y)
        
    model_public_vs_residential = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    # RETAIL vs RESIDENTIAL
    data = pd.concat([_data,_labels], axis=1)
    data_1 = data[data.CLASE == "RETAIL"]
    data_2 = data[data.CLASE == "RESIDENTIAL"]
    data = pd.concat([data_1, data_2])
    labels = data.CLASE
    data = data.drop('CLASE', axis=1)
    X = data.values
    y = labels.values
    
    if(under):
        sampling_dict = {'RESIDENTIAL': n_us}
        us = RandomUnderSampler(random_state=42, sampling_strategy=sampling_dict) 
        X, y = us.fit_resample(X, y)
    
    model_retail_vs_residential = KNeighborsClassifier(n_jobs=-1, n_neighbors=10, weights='distance', metric='manhattan').fit(X,y)
    
    print("FIT MODELS: DONE")
    
    # Diccionario que guarda las prob de cada modelo
    dict_prob = {}
    
    # Diccionario clases:
    dict_class = {
        'OFFICEvsRETAIL':['OFFICE','RETAIL'],
        'OFFICEvsPUBLIC':['OFFICE','PUBLIC'],
        'OFFICEvsOTHER':['OFFICE','OTHER'],
        'OFFICEvsINDUSTRIAL':['INDUSTRIAL','OFFICE'],
        'OFFICEvsAGRICULTURE':['AGRICULTURE','OFFICE'],
        'OFFICEvsRESIDENTIAL':['OFFICE','RESIDENTIAL'],
        'AGRICULTUREvsRETAIL':['AGRICULTURE','RETAIL'],
        'AGRICULTUREvsPUBLIC':['AGRICULTURE','PUBLIC'],
        'AGRICULTUREvsOTHER':['AGRICULTURE','OTHER'],
        'AGRICULTUREvsINDUSTRIAL':['AGRICULTURE','INDUSTRIAL'],
        'AGRICULTUREvsRESIDENTIAL':['AGRICULTURE','RESIDENTIAL'],
        'INDUSTRIALvsRETAIL':['INDUSTRIAL','RETAIL'],
        'INDUSTRIALvsPUBLIC':['INDUSTRIAL','PUBLIC'],
        'INDUSTRIALvsOTHER':['INDUSTRIAL','OTHER'],
        'INDUSTRIALvsRESIDENTIAL':['INDUSTRIAL','RESIDENTIAL'],
        'OTHERvsRETAIL':['OTHER','RETAIL'],
        'OTHERvsPUBLIC':['OTHER','PUBLIC'],
        'OTHERvsRESIDENTIAL':['OTHER','RESIDENTIAL'],
        'PUBLICvsRETAIL':['PUBLIC','RETAIL'],
        'PUBLICvsRESIDENTIAL':['PUBLIC','RESIDENTIAL'],
        'RETAILvsRESIDENTIAL':['RESIDENTIAL','RETAIL']
    }

    # Predicción de las probabilidades:

    # => Sin selección/creación de características:
    
    dict_prob['OFFICEvsRETAIL'] = model_office_vs_retail.predict_proba(test.values)
    dict_prob['OFFICEvsPUBLIC'] = model_office_vs_public.predict_proba(test.values)
    dict_prob['OFFICEvsOTHER'] = model_office_vs_other.predict_proba(test.values)
    dict_prob['OFFICEvsINDUSTRIAL'] = model_office_vs_industrial.predict_proba(test.values)
    dict_prob['OFFICEvsAGRICULTURE'] = model_office_vs_agriculture.predict_proba(test.values)
    dict_prob['OFFICEvsRESIDENTIAL'] = model_office_vs_residential.predict_proba(test.values)
    dict_prob['AGRICULTUREvsRETAIL'] = model_agriculture_vs_retail.predict_proba(test.values)
    dict_prob['AGRICULTUREvsPUBLIC'] = model_agriculture_vs_public.predict_proba(test.values)
    dict_prob['AGRICULTUREvsOTHER'] = model_agriculture_vs_other.predict_proba(test.values)
    dict_prob['AGRICULTUREvsINDUSTRIAL'] = model_agriculture_vs_industrial.predict_proba(test.values)
    dict_prob['AGRICULTUREvsRESIDENTIAL'] = model_agriculture_vs_residential.predict_proba(test.values)
    dict_prob['INDUSTRIALvsRETAIL'] = model_industrial_vs_retail.predict_proba(test.values)
    dict_prob['INDUSTRIALvsPUBLIC'] = model_industrial_vs_public.predict_proba(test.values)
    dict_prob['INDUSTRIALvsOTHER'] = model_industrial_vs_other.predict_proba(test.values)
    dict_prob['INDUSTRIALvsRESIDENTIAL'] = model_industrial_vs_residential.predict_proba(test.values)
    dict_prob['OTHERvsRETAIL'] = model_other_vs_retail.predict_proba(test.values)
    dict_prob['OTHERvsPUBLIC'] = model_other_vs_public.predict_proba(test.values)
    dict_prob['OTHERvsRESIDENTIAL'] = model_other_vs_residential.predict_proba(test.values)
    dict_prob['PUBLICvsRETAIL'] = model_public_vs_retail.predict_proba(test.values)
    dict_prob['PUBLICvsRESIDENTIAL'] = model_public_vs_residential.predict_proba(test.values)
    dict_prob['RETAILvsRESIDENTIAL'] = model_retail_vs_residential.predict_proba(test.values)

    # => Con selección/creación de características:

#     aux = test.copy()
#     aux = aux.drop('GEOM_R2', axis=1)
#     aux['MAXBUILDINGFLOOR2'] = aux['MAXBUILDINGFLOOR']**2
#     aux['Q_G_3_0_32'] = aux['Q_G_3_0_3']**2
#     dict_prob['OFFICEvsRETAIL'] = model_office_vs_retail.predict_proba(aux.values)

#     aux = test.copy()
#     aux = aux.drop('GEOM_R3', axis=1)
#     aux['Y2'] = aux['Y']**2
#     aux['Q_NIR_8_0_92'] = aux['Q_NIR_8_0_9']**2
#     aux['Q_B_2_0_32'] = aux['Q_B_2_0_3']**2
#     dict_prob['OFFICEvsPUBLIC'] = model_office_vs_public.predict_proba(aux.values)

#     aux = test.copy()
#     aux = aux.drop('Q_B_2_0_2', axis=1)
#     aux = aux.drop('Q_B_2_1_0', axis=1)
#     aux = aux.drop('Q_NIR_8_0_8', axis=1)
#     aux['CADASTRALQUALITYID2'] = aux['CADASTRALQUALITYID']**2
#     aux['Q_NIR_8_0_52'] = aux['Q_NIR_8_0_5']**2
#     aux['Q_G_3_0_22'] = aux['Q_G_3_0_2']**2
#     dict_prob['OFFICEvsOTHER'] = model_office_vs_other.predict_proba(aux.values)

#     aux = test.copy()
#     aux = aux.drop('Q_NIR_8_0_8', axis=1)
#     aux['Q_B_2_0_62'] = aux['Q_B_2_0_6']**2
#     dict_prob['OFFICEvsINDUSTRIAL'] = model_office_vs_industrial.predict_proba(aux.values)

#     aux = test.copy()
#     aux = aux.drop('Q_G_3_0_2', axis=1)
#     aux['Y2'] = aux['Y']**2
#     aux['X2'] = aux['X']**2
#     aux['Q_NIR_8_0_32'] = aux['Q_NIR_8_0_3']**2
#     dict_prob['OFFICEvsAGRICULTURE'] = model_office_vs_agriculture.predict_proba(aux.values)

#     dict_prob['OFFICEvsRESIDENTIAL'] = model_office_vs_residential.predict_proba(test.values)

#     dict_prob['AGRICULTUREvsRETAIL'] = model_agriculture_vs_retail.predict_proba(test.values)

#     dict_prob['AGRICULTUREvsPUBLIC'] = model_agriculture_vs_public.predict_proba(test.values)

#     dict_prob['AGRICULTUREvsOTHER'] = model_agriculture_vs_other.predict_proba(test.values)

#     dict_prob['AGRICULTUREvsINDUSTRIAL'] = model_agriculture_vs_industrial.predict_proba(test.values)

#     dict_prob['AGRICULTUREvsRESIDENTIAL'] = model_agriculture_vs_residential.predict_proba(test.values)

#     dict_prob['INDUSTRIALvsRETAIL'] = model_industrial_vs_retail.predict_proba(test.values)

#     dict_prob['INDUSTRIALvsPUBLIC'] = model_industrial_vs_public.predict_proba(test.values)

#     dict_prob['INDUSTRIALvsOTHER'] = model_industrial_vs_other.predict_proba(test.values)

#     dict_prob['INDUSTRIALvsRESIDENTIAL'] = model_industrial_vs_residential.predict_proba(test.values)

#     dict_prob['OTHERvsRETAIL'] = model_other_vs_retail.predict_proba(test.values)

#     dict_prob['OTHERvsPUBLIC'] = model_other_vs_public.predict_proba(test.values)

#     dict_prob['OTHERvsRESIDENTIAL'] = model_other_vs_residential.predict_proba(test.values)

#     dict_prob['PUBLICvsRETAIL'] = model_public_vs_retail.predict_proba(test.values)

#     dict_prob['PUBLICvsRESIDENTIAL'] = model_public_vs_residential.predict_proba(test.values)

#     dict_prob['RETAILvsRESIDENTIAL'] = model_retail_vs_residential.predict_proba(test.values)

    print("PREDICT: DONE")
    
    # ====== AGREGACIÓN: ====== #
    pred = []
    
    for i in range(0, test.shape[0]):

        dict_count = {
            'OFFICE': 0,
            'AGRICULTURE': 0,
            'INDUSTRIAL': 0,
            'OTHER': 0,
            'PUBLIC': 0,
            'RETAIL': 0,
            'RESIDENTIAL': 0
        }
        max_prob = 0
        for key in dict_prob.keys():
            
            # ==> VOTO SIMPLE:
#             if (dict_prob[key][i][0]>dict_prob[key][i][1]):
#                 dict_count[dict_class[key][0]] +=1
#             else:
#                 dict_count[dict_class[key][1]] +=1

            # ==> SUMA PROB DE CADA CLASE:
            dict_count[dict_class[key][0]] += dict_prob[key][i][0]
            dict_count[dict_class[key][1]] += dict_prob[key][i][1]

            # ==> PROB VENCEDORA - PROB PERDEDORA:
#             if (dict_prob[key][i][0]>dict_prob[key][i][1]):
#                 dict_count[dict_class[key][0]] += dict_prob[key][i][0]-dict_prob[key][i][1]
#             else:
#                 dict_count[dict_class[key][1]] += dict_prob[key][i][1]-dict_prob[key][i][0]
            
            # ==> PROB MAX:
#             x = dict_prob[key][i][0]
#             y = dict_prob[key][i][1]
#             if (x>y):
#                 if(x>max_prob):
#                     max_prob = x
#                     clase = dict_class[key][0]
#             else:
#                 if(y>max_prob):
#                     max_prob = y
#                     clase = dict_class[key][1]
        
        predicted_class = max(dict_count.items(), key=operator.itemgetter(1))[0]
        pred.append(predicted_class)
        #pred.append(clase)
    
    return(pred)